## ECGR 4105 - Homework 6

## Problem 1, Part a

For 1st Part, a Fully Connected Neural Network for all 10 classes in CIFAR-10 will be created with only one hidden layer with the size of 512. The Network will be trained for 300 epochs. Training Time, Training Loss, and Evaluation Accuracy after 300 epochs will be reported.

In [49]:
# Importing all the necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importing Pytorch and Tensorflow
import torch
torch.version.__version__
import torch.optim as optim
import torch.nn as nn

In [2]:
# Importing/Downloading the CIFAR-10 DataSet

from torchvision import datasets
data_path = "../data-unversioned/p1ch7/"
cifar10 = datasets.CIFAR10(data_path, train = True, download = True)
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
print("Training Set: ", len(cifar10))
print("Validation Set: ", len(cifar10_val))

Training Set:  50000
Validation Set:  10000


In [4]:
print(cifar10)
print(cifar10_val)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../data-unversioned/p1ch7/
    Split: Train
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ../data-unversioned/p1ch7/
    Split: Test


In [5]:
# Convertion of the PIL Images into PyTorch Tensors
from torchvision import transforms

tensor_cifar10 = datasets.CIFAR10(data_path, train = True, download = False,
                                  transform = transforms.ToTensor())

tensor_cifar10_val = datasets.CIFAR10(data_path, train = False, download = False,
                                      transform = transforms.ToTensor())

In [6]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim = 3)
imgs.shape

print(imgs.view(3, -1).mean(dim = 1))
print(imgs.view(3, -1).std(dim = 1))

tensor([0.4914, 0.4822, 0.4465])
tensor([0.2470, 0.2435, 0.2616])


In [7]:
imgs_val = torch.stack([img_t_val for img_t_val, _ in tensor_cifar10_val], dim = 3)
imgs_val.shape

print(imgs_val.view(3, -1).mean(dim = 1))
print(imgs_val.view(3, -1).std(dim = 1))

tensor([0.4942, 0.4851, 0.4504])
tensor([0.2467, 0.2429, 0.2616])


In [8]:
transformed_cifar10 = datasets.CIFAR10(data_path, train = True, download = False,

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
]))

In [9]:
transformed_cifar10_val = datasets.CIFAR10(data_path, train = False, download = False,

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4942, 0.4851, 0.4404),
                         (0.2467, 0.2429, 0.2616))
]))

In [10]:
#Importing 'datetime' Module included with Python
import datetime

In [43]:
# Training the Model (with only 1 Hidden Layer)

train_loader_part_a = torch.utils.data.DataLoader(transformed_cifar10, batch_size = 128, shuffle = True)

loaded_model_part_a = nn.Sequential(nn.Linear(3072, 512), nn.Tanh(),
                      nn.Linear(512, 10),
                      nn.LogSoftmax(dim = 1))
learning_rate = 1e-2

optimizer = optim.SGD(loaded_model_part_a.parameters(), lr = learning_rate)

loss_Fn = nn.NLLLoss()

n_epochs = 300

for epoch in range(n_epochs):
    for imgs, labels in train_loader_part_a:
        batch_size = imgs.shape[0]
        outputs = loaded_model_part_a(imgs.view(batch_size, -1))
        loss = loss_Fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(epoch % 10 == 0):
        print("{} Epoch {}, Training Loss {}".format(datetime.datetime.now(), epoch, float(loss)))

2022-12-08 08:02:46.473520 Epoch 0, Training Loss 1.781964898109436


KeyboardInterrupt: 

In [ ]:
torch.save(model_part_a.state_dict(), data_path + "Problem1_PartA.pt")

In [45]:
# Computing the Accuracy of the Model using the Validation Set 

val_loader_part_a = torch.utils.data.DataLoader(transformed_cifar10_val, batch_size = 128, shuffle = False)

# Initializing the values of the Variables "correct"" and "total" for further use
correct = 0;
total = 0;

with torch.no_grad():
    for imgs_val, labels in val_loader_part_a:
        batch_size = imgs_val.shape[0]
        outputs = loaded_model_part_a(imgs_val.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim = 1)
        
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print("Accuracy: ", correct / total)

Accuracy:  0.4195


In [41]:
torch.save(model_part_a.state_dict(), data_path + "Problem1_PartA.pt")
loaded_model_part_a = nn.Sequential(nn.Linear(3072, 512), nn.Tanh(),
                      nn.Linear(512, 10),
                      nn.LogSoftmax(dim = 1))
loaded_model_part_a.load_state_dict(torch.load(data_path + "Problem1_PartA.pt"))

<All keys matched successfully>

## Problem 1, Part b

For 2nd Part, the Network will be extended with two more additional hidden layers. The Network will be trained for 300 epochs and the Training Time, Training Loss, and Evaluation Accuracy after 300 epochs will be reported.

In [14]:
# Training the Model (with 2 Additional Hidden Layer)

train_loader_part_b = torch.utils.data.DataLoader(transformed_cifar10, batch_size = 64, shuffle = True)

model_part_b = nn.Sequential(nn.Linear(3072, 2048), nn.Tanh(),
                      nn.Linear(2048, 512), nn.Tanh(),
                      nn.Linear(512, 256), nn.Tanh(),
                      nn.Linear(256, 10), nn.Tanh(),
                      nn.LogSoftmax(dim = 1))
learning_rate = 1e-2

optimizer = optim.SGD(model_part_b.parameters(), lr = learning_rate)

loss_Fn = nn.NLLLoss()

n_epochs = 300

for epoch in range(n_epochs):
    for imgs, labels in train_loader_part_b:
        batch_size = imgs.shape[0]
        outputs = model_part_b(imgs.view(batch_size, -1))
        loss = loss_Fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(epoch % 10 == 0):
       print("{} Epoch {}, Training Loss {}".format(datetime.datetime.now(), epoch, float(loss)))

2022-12-08 01:41:47.258438 Epoch 0, Training Loss 2.0479979515075684
2022-12-08 01:50:34.266069 Epoch 10, Training Loss 1.6230140924453735
2022-12-08 01:59:16.812208 Epoch 20, Training Loss 1.342703104019165
2022-12-08 02:07:58.768081 Epoch 30, Training Loss 1.2078759670257568
2022-12-08 02:16:43.894495 Epoch 40, Training Loss 1.450156807899475
2022-12-08 02:25:26.807374 Epoch 50, Training Loss 0.9796058535575867
2022-12-08 02:34:10.600114 Epoch 60, Training Loss 0.9214929938316345
2022-12-08 02:42:56.486086 Epoch 70, Training Loss 1.1126482486724854
2022-12-08 02:51:40.032863 Epoch 80, Training Loss 1.0062534809112549
2022-12-08 03:00:21.689096 Epoch 90, Training Loss 0.9504820108413696
2022-12-08 03:09:02.445550 Epoch 100, Training Loss 1.1079751253128052
2022-12-08 03:17:43.685620 Epoch 110, Training Loss 0.8577156662940979
2022-12-08 03:26:23.536683 Epoch 120, Training Loss 1.0073959827423096
2022-12-08 03:35:04.459506 Epoch 130, Training Loss 0.9235573410987854
2022-12-08 03:43:54

In [51]:
# Computing the Accuracy of the using the Validation Set 

val_loader = torch.utils.data.DataLoader(transformed_cifar10_val, batch_size = 128, shuffle = False)

# Initializing the values of the Variables "correct"" and "total" for further use
correct = 0;
total = 0;

with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim = 1)
        
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print("Accuracy: %f", correct / total)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [128, 3072]

In [44]:
torch.save(model_part_b.state_dict(), data_path + "Problem1_PartB.pt")